In [2]:
from src.utils.data_utils import load_dataframe_from_csv
from src.data.dataloader import load_initial_dataset
from src.utils.clean_cmu import clean_movies_cmu
from config import *

import plotly.express as px
import pandas as pd
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

In [3]:
movie_cmu = load_dataframe_from_csv('movie_cmu.csv')
character = load_dataframe_from_csv('character.csv')
plot_summaries = load_dataframe_from_csv('plot_summaries.csv')
tvtropes = load_dataframe_from_csv('tvtropes.csv')
name_clusters = load_dataframe_from_csv('name_clusters.csv')
oscar_winning_films = load_dataframe_from_csv('oscar_winning_films_ids.csv')
oscar_winning_actors = load_dataframe_from_csv('oscar_winning_actors.csv')
oscar_winning_actresses = load_dataframe_from_csv('oscar_winning_actresses.csv')
oscar_supporting_actors = load_dataframe_from_csv('oscar_winning_supporting_actors.csv')
oscar_supporting_actresses = load_dataframe_from_csv('oscar_winning_supporting_actresses.csv')
extended_films = load_dataframe_from_csv('film_2015_2024.csv')
academy_award_winning_films = load_dataframe_from_csv('acedemy_award_winning_films')
winning_actors_info = load_dataframe_from_csv('winning_actors_information.csv')
imdb_ratings = load_dataframe_from_csv('imdb_ratings.csv')
oscar_actors_info = load_dataframe_from_csv('oscar_actors_info.csv')
oscar_actress_info = load_dataframe_from_csv('oscar_actress_info.csv')
new_film_dataset = load_dataframe_from_csv('film_full_2.csv')

d:\UNIV\MASTER-EPFL\ADA\applied-project\ada-2024-project-5ds\src\utils\data_utils.py:27: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loaded = pd.read_csv(csv_file_path)


In [4]:
new_film_dataset.columns

Index(['wikipedia_id', 'dataset_id', 'title', 'release_date', 'box_office',
       'runtime', 'languages', 'countries', 'genres', 'reviewScores',
       'awardsReceived', 'awardsNominated', 'capitalCost', 'release date',
       'categories', 'nbOscarReceived', 'nbOscarNominated'],
      dtype='object')

In [5]:
extended_films["Movie genres"] = extended_films["genres"]
extended_films["Movie release date"] = extended_films["release_date"]
extended_films["Movie languages"] = extended_films["languages"]
extended_films["Movie countries"] = extended_films["countries"]
extended_films["Movie name"] = extended_films["film"]
extended_films["Movie box office revenue"] = extended_films["box_office"]
extended_films["Movie runtime"] = extended_films["runtime"]
extended_films["Wikipedia movie ID"] = extended_films["page_id"]
extended_films = extended_films.drop(columns=["genres", "release_date", "languages", "countries", "release date", "film", "box_office", "runtime", "page_id"])

winning_actors_info["Actor name"] = winning_actors_info["actor"]
winning_actors_info = winning_actors_info.drop(columns=["actor"])

imdb_ratings['Movie release date'] = imdb_ratings['title_year']  
imdb_ratings['Movie name'] = imdb_ratings['movie_title'] 
imdb_ratings = imdb_ratings.drop(columns=['title_year', 'movie_title'])

new_film_dataset['Wikipedia movie ID'] = new_film_dataset['wikipedia_id']
new_film_dataset['Movie name'] = new_film_dataset['title']
new_film_dataset['Movie release date'] = new_film_dataset['release_date']
new_film_dataset['Movie genres'] = new_film_dataset['categories']
new_film_dataset['Movie box office revenue'] = new_film_dataset['box_office']

new_film_dataset.drop(columns=['wikipedia_id', 'title', 'release_date', 'categories', 'box_office'], inplace=True)



In [6]:
oscar_act = pd.concat([oscar_winning_actors, oscar_winning_actresses], axis=0)
oscar_act_movies_cmu  = pd.merge(oscar_act, movie_cmu, left_on='film_id', right_on='Wikipedia movie ID')
oscar_act_movies_cmu.drop(columns=['film_id'], inplace=True) 

print("CMU: ")
print(oscar_act_movies_cmu.shape)
print(oscar_act_movies_cmu['Movie release date'].min())
print(oscar_act_movies_cmu['Movie release date'].max())


CMU: 
(174, 11)
1928.0
2012.0


In [7]:
oscar_act_movies_extendedDS  = pd.merge(oscar_act, extended_films, left_on='film_id', right_on='Wikipedia movie ID')
oscar_act_movies_extendedDS.drop(columns=['film_id'], inplace=True) 

# Convert the 'Movie release date' column to datetime
oscar_act_movies_extendedDS['Movie release date'] = pd.to_datetime(oscar_act_movies_extendedDS['Movie release date'])
# Format the column to only show Year-Month-Day
oscar_act_movies_extendedDS['Movie release date'] = oscar_act_movies_extendedDS['Movie release date'].dt.strftime('%Y-%m-%d')

print("Extended DS")
print(oscar_act_movies_extendedDS.shape)
print(oscar_act_movies_extendedDS['Movie release date'].min())
print(oscar_act_movies_extendedDS['Movie release date'].max())

Extended DS
(16, 15)
2015-09-04
2023-09-01


In [8]:
oscar_act_movies = pd.concat([oscar_act_movies_cmu, oscar_act_movies_extendedDS], axis=0)
oscar_act_movies.drop(columns=['wikidata_id'], inplace=True)
print(oscar_act_movies.shape)
print(oscar_act_movies.columns)

(190, 15)
Index(['Actors id', 'Actress id', 'Wikipedia movie ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres',
       'reviewScores', 'awardsReceived', 'awardsNominated', 'capitalCost'],
      dtype='object')


In [9]:
oscar_act_movies_new = pd.merge(oscar_act, new_film_dataset, left_on='film_id', right_on='Wikipedia movie ID')
oscar_act_movies_new.drop(columns=['film_id'], inplace=True)


print("film_full")
print(oscar_act_movies_new.shape)
print(oscar_act_movies_new['Movie release date'].min())
print(oscar_act_movies_new['Movie release date'].max())
oscar_act_movies_new.columns

film_full
(190, 19)
1927-01-01
2023-09-01


Index(['Actors id', 'Actress id', 'dataset_id', 'runtime', 'languages',
       'countries', 'genres', 'reviewScores', 'awardsReceived',
       'awardsNominated', 'capitalCost', 'release date', 'nbOscarReceived',
       'nbOscarNominated', 'Wikipedia movie ID', 'Movie name',
       'Movie release date', 'Movie genres', 'Movie box office revenue'],
      dtype='object')

In [10]:
oscar_actress_info_copy = pd.merge(winning_actors_info, oscar_act_movies_new, left_on='page_id', right_on='Actress id')
oscar_actors_info_copy = pd.merge(winning_actors_info, oscar_act_movies_new, left_on='page_id', right_on='Actors id')
oscar_act_info = pd.concat([oscar_actors_info_copy, oscar_actress_info_copy], axis=0)
# Check the type and standardize the date format
def standardize_date_format(date):
    if pd.isna(date):  # Handle NaN values explicitly
        return None  # Or return a default value, e.g., -1 or an empty string
    if isinstance(date, str):  # If it's a string (datetime-like), process it
        return pd.to_datetime(date).year  # Extract just the year from the datetime
    elif isinstance(date, float):  # If it's a float, treat it as a year
        return int(date)  # Convert float year to integer year
    return None  # Handle missing or invalid data

# Apply the function to create a new column with unified format
oscar_act_info['Movie release year'] = oscar_act_info['Movie release date'].apply(standardize_date_format)

In [11]:
print(oscar_act_info.columns)
oscar_act_info.drop(columns=['sexLabel'], inplace=True)  #Remove the columns that are in character.csv
oscar_act_info.head(2)
print(oscar_act_info.shape)

Index(['page_id', 'wikidata_id', 'sexLabel', 'nativeLanguageLabel',
       'countryOfCitizenshipLabel', 'ethnicGroupLabel', 'Actor name',
       'Actors id', 'Actress id', 'dataset_id', 'runtime', 'languages',
       'countries', 'genres', 'reviewScores', 'awardsReceived',
       'awardsNominated', 'capitalCost', 'release date', 'nbOscarReceived',
       'nbOscarNominated', 'Wikipedia movie ID', 'Movie name',
       'Movie release date', 'Movie genres', 'Movie box office revenue',
       'Movie release year'],
      dtype='object')
(190, 26)


In [12]:
duplicates_with_different_ages = oscar_act_info.groupby('Actor name').filter(lambda x: x['page_id'].nunique() > 1)

if not duplicates_with_different_ages.empty:
    print("Duplicate names with different ids found:")
    print(duplicates_with_different_ages)
else:
    print("No duplicate names with different Actor/Actress ids found.")





No duplicate names with different Actor/Actress ids found.


In [13]:
# Prepare data for box office revenue evolution
oscar_revenue_data_corrected = oscar_act_info[['Movie box office revenue', 'Movie release year']].dropna()
oscar_revenue_data_corrected = oscar_revenue_data_corrected.dropna(subset=['Movie release year'])

# Group by year and calculate total box office revenue
oscar_revenue_data_corrected['Year'] = oscar_revenue_data_corrected['Movie release year']
revenue_by_year_corrected = oscar_revenue_data_corrected.groupby('Year')['Movie box office revenue'].sum().reset_index()

# Create a line chart
fig = px.line(
    revenue_by_year_corrected,
    x='Year',
    y='Movie box office revenue',
    title='Evolution of Movie Box Office Revenue Over the Years',
    labels={'Year': 'Year', 'Movie box office revenue': 'Box Office Revenue ($)'},
    line_shape='spline'
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='Total Box Office Revenue ($)', tickformat='$,.0f'),
    width=1400, 
    height=600  
)

fig.show()

In [14]:
oscar_act_info = pd.merge(oscar_act_info, character[['Wikipedia movie ID', 'Actor name', 'Actor age at movie release', 'Actor date of birth', 'Actor ethnicity','Character name', 'Freebase actor ID']], on=['Wikipedia movie ID', 'Actor name'])


In [15]:
print('Shape:', oscar_act_info.shape)
print('Columns:', oscar_act_info.columns)
missing_values = oscar_winning_actresses['Actress id'][~oscar_winning_actresses['Actress id'].isin(oscar_act_info['Actress id'])]

Shape: (177, 31)
Columns: Index(['page_id', 'wikidata_id', 'nativeLanguageLabel',
       'countryOfCitizenshipLabel', 'ethnicGroupLabel', 'Actor name',
       'Actors id', 'Actress id', 'dataset_id', 'runtime', 'languages',
       'countries', 'genres', 'reviewScores', 'awardsReceived',
       'awardsNominated', 'capitalCost', 'release date', 'nbOscarReceived',
       'nbOscarNominated', 'Wikipedia movie ID', 'Movie name',
       'Movie release date', 'Movie genres', 'Movie box office revenue',
       'Movie release year', 'Actor age at movie release',
       'Actor date of birth', 'Actor ethnicity', 'Character name',
       'Freebase actor ID'],
      dtype='object')


In [16]:
if 'Actor age at movie release' in oscar_act_info.columns:
    # Count occurrences of each age
    age_distribution = oscar_act_info['Actor age at movie release'].value_counts().reset_index()
    age_distribution.columns = ['Actor age at movie release', 'count']
    
    # Sort by age for better visualization
    age_distribution = age_distribution.sort_values(by='Actor age at movie release')

    # Create the bar plot
    fig = px.bar(age_distribution, x='Actor age at movie release', y='count', title='Age Distribution', labels={'age': 'Age', 'count': 'Count'})
    fig.show()
else:
    print("The column 'age' does not exist in the provided file.")

In [17]:
# Prepare data for men, women, and both
# Assuming `age_distribution` contains combined data, and separate dataframes exist for men and women
men = oscar_actors_info['Actor age at movie release'].value_counts().reset_index()
women = oscar_actress_info['Actor age at movie release'].value_counts().reset_index()

# Rename columns for consistency
men.columns = ['Age', 'Actors']
women.columns = ['Age', 'Actresses']

# Merge all into a single dataframe for plotting
merged = men.merge(women, on='Age', how='outer').fillna(0)

# Melt the data for easier plotting with Plotly
melted = merged.melt(id_vars='Age', value_vars=['Actors', 'Actresses'], 
                     var_name='Group', value_name='Count')

# Plot grouped bar chart
fig = px.bar(melted, x='Age', y='Count', color='Group', barmode='group',
             labels={'Age': 'Age', 'Count': 'Count', 'Group': 'Group'},
             title='Age Distribution')
fig.show()

In [18]:
# Print all unique values in the 'Actor ethnicity' column
unique_ethnicities = oscar_act_info['Actor ethnicity'].unique()

# Display the unique values
print(unique_ethnicities)

[nan 'Jewish people' 'Irish Americans' 'Scottish Americans'
 'Puerto Ricans' 'Dutch Americans' 'Swiss' 'African Americans'
 'Scotch-Irish Americans' 'White people' 'White Americans'
 'Italian Americans' 'Slovak Americans' 'Indians' 'English people'
 'Irish migration to Great Britain' 'Portuguese Americans'
 'German Americans' 'Italians' 'Māori' 'Hungarian Americans' 'Akan people'
 'British' 'Canadian Americans' 'British Americans' 'Swedish Americans'
 'Irish people' 'American Jews' 'White British' 'Germans'
 'Czech Americans' 'Armenians' 'English Americans' 'Spanish Americans'
 'White Africans of European ancestry' 'French' 'Israeli Americans']


In [19]:
country_counts = oscar_act_info['countryOfCitizenshipLabel'].value_counts()
country_counts = country_counts.reset_index()
country_counts.columns = ['Country', 'Count']

fig = px.choropleth(country_counts, 
                    locations="Country", 
                    locationmode='country names', 
                    color="Count", 
                    hover_name="Country", 
                    title="Heatmap of Oscar Winning Actors by Country")
fig.update_geos(showcountries=True)
fig.show()

In [20]:
country_counts = oscar_act_info['countryOfCitizenshipLabel'].value_counts()
country_counts = country_counts.reset_index()
country_counts.columns = ['Country', 'Count']

fig = px.choropleth(country_counts, 
                    locations="Country", 
                    locationmode='country names', 
                    color="Count", 
                    hover_name="Country", 
                    title="Heatmap of Oscar Winning Actors by Country")
fig.update_geos(showcountries=True, visible=True)
fig.update_geos(lonaxis_range=[-10, 40], lataxis_range=[35, 70])
fig.show()

In [21]:
print('Average Capital Cost of the movies starring the actresses:', oscar_act_movies['capitalCost'].median()) 
print('Average Box Office revenue of the movies starring the actresses:', oscar_act_movies['Movie box office revenue'].median())
print('Average Benefits of the movies starring the actresses:', (oscar_act_movies['Movie box office revenue'] - oscar_act_movies['capitalCost']).median())


Average Capital Cost of the movies starring the actresses: 25000000.0
Average Box Office revenue of the movies starring the actresses: 41461428.0
Average Benefits of the movies starring the actresses: 96792237.0


In [22]:
oscar_act_movies_all = character[character['Freebase actor ID'].isin(oscar_act_info['Freebase actor ID'])]
oscar_act_movies_all = oscar_act_movies_all.drop(columns=['Freebase movie ID', 'Movie release date'])
oscar_act_movies_all = oscar_act_movies_all.merge(movie_cmu, on='Wikipedia movie ID', how='left')
print('Shape:', oscar_act_movies_all.shape)
print('Columns:', oscar_act_movies_all.columns)
print('Date boundary:', oscar_act_movies_all['Movie release date'].min(), '-', oscar_act_movies_all['Movie release date'].max())

Shape: (6867, 19)
Columns: Index(['Wikipedia movie ID', 'Character name', 'Actor date of birth',
       ' Actor gender', 'Actor height)', 'Actor ethnicity', 'Actor name',
       'Actor age at movie release', 'Freebase character',
       'Freebase character ID', 'Freebase actor ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres'],
      dtype='object')
Date boundary: 1909.0 - 2014.0


In [23]:
# Create a set of unique (Actor name, Wikipedia movie ID) combinations for rewards
best_act_movies = set(
    zip(oscar_act_info['Actor name'], oscar_act_info['Wikipedia movie ID'])
)

# Add a new column indicating if the movie is a Best Actress Reward
oscar_act_movies_all['Best Actress Reward'] = oscar_act_movies_all.apply(
    lambda row: (row['Actor name'], row['Wikipedia movie ID']) in best_act_movies,
    axis=1
)

In [24]:
# Prepare the data for the bar chart
oscar_movies_all = oscar_act_movies_all.copy()
oscar_movies_all['Movie genres'] = oscar_movies_all['Movie genres'].fillna('Unknown')
oscar_movies_all['Best Actress Reward'] = oscar_movies_all['Best Actress Reward'].map({True: 'Oscar-Winning', False: 'Non-Oscar-Winning'})

# Split the genres into individual rows for analysis
genres_split = oscar_movies_all.assign(Movie_genres_split=oscar_movies_all['Movie genres'].str.split(', ')).explode('Movie_genres_split')

# Group and count movies by genre and whether they won an Oscar
genre_counts = genres_split.groupby(['Movie_genres_split', 'Best Actress Reward']).size().reset_index(name='Count')

# Limit to the top 15 genres with the highest total count
top_genres = genre_counts.groupby('Movie_genres_split')['Count'].sum().nlargest(15).index
filtered_genre_counts = genre_counts[genre_counts['Movie_genres_split'].isin(top_genres)]

# Create the improved bar chart with log scale
fig = px.bar(
    filtered_genre_counts.sort_values('Count', ascending=False),
    x='Movie_genres_split',
    y='Count',
    color='Best Actress Reward',
    title='Top 15 Movie Genres (Oscar-Winning vs Non-Oscar-Winning)',
    labels={'Movie_genres_split': 'Movie Genre', 'Count': 'Number of Movies'},
    barmode='group',
    log_y=True  # Set y-axis to log scale
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(tickangle=45, title='Movie Genre'),
    yaxis=dict(title='Number of Movies (Log Scale)'),
    width=1000,  # Increase figure width
    height=500   # Adjust height
)

# Uncomment this to visualize
# fig.show()

In [25]:
# Prepare data for box office revenue comparison
print(oscar_movies_all.columns)
box_office_data = oscar_act_movies_all[['Movie box office revenue', 'Best Actress Reward']].dropna()

# Create a box plot to show the distribution of box office revenue
fig = px.box(
    box_office_data,
    x='Best Actress Reward',
    y='Movie box office revenue',
    title='Box Office Revenue Distribution (Oscar-Winning vs Non-Oscar-Winning)',
    labels={'Best Actress Reward': 'Movie Type', 'Movie box office revenue': 'Box Office Revenue ($)'},
    color='Best Actress Reward'
)

# Update layout for better readability⁄
fig.update_layout(
    xaxis=dict(title='Best Actress Reward'),
    yaxis=dict(title='Box Office Revenue ($)', tickformat='$,.0f'),
    width=800,  # Adjust figure width
    height=600  # Adjust figure height
)

# Uncomment this to visualize
fig.show()

Index(['Wikipedia movie ID', 'Character name', 'Actor date of birth',
       ' Actor gender', 'Actor height)', 'Actor ethnicity', 'Actor name',
       'Actor age at movie release', 'Freebase character',
       'Freebase character ID', 'Freebase actor ID', 'Freebase movie ID',
       'Movie name', 'Movie release date', 'Movie box office revenue',
       'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres',
       'Best Actress Reward'],
      dtype='object')


In [26]:
scatter_data = oscar_act_info[['Actor age at movie release', 'Movie box office revenue']].dropna()


fig = px.scatter(
    scatter_data,
    x='Actor age at movie release',
    y='Movie box office revenue',
    title='Actors Age vs. Movie Box Office Revenue (Oscar-Winning Movies)',
    labels={
        'Actor age at movie release': 'Age at Movie Release',
        'Movie box office revenue': 'Box Office Revenue ($)'
    },
    size='Movie box office revenue',
    hover_data=['Actor age at movie release'],
    color_discrete_sequence=['purple']
)


fig.update_layout(
    xaxis=dict(title='Age at Movie Release'),
    yaxis=dict(title='Box Office Revenue ($)', tickformat='$,.0f'),
    width=800,  
    height=600  
)

fig.show()

In [27]:
import re

# Extract and clean the genres
genres_data = oscar_movies_all[['Movie genres']].dropna()
genres_data = genres_data.assign(Movie_genres_split=genres_data['Movie genres'].str.split(', ')).explode('Movie_genres_split')

# Use regex to remove any empty strings, square brackets, and extra spaces
genres_data['Movie_genres_split'] = genres_data['Movie_genres_split'].apply(lambda x: re.sub(r"[\[\]']", '', x).strip())

# Count the occurrences of each genre
genre_counts = genres_data['Movie_genres_split'].value_counts().reset_index()
genre_counts.columns = ['Genre', 'Count']
print(genre_counts)

# Plot the top 10 genres
fig = px.pie(
    genre_counts.head(10), 
    values='Count',
    title='Top 10 Genres in Best Actor/Actress Academy Award Movies',
    labels={'Genre': 'Movie Genre', 'Count': 'Number of Movies'},
    names='Genre',
    color='Genre'
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Movie Genre', tickangle=45),
    yaxis=dict(title='Number of Movies'),
    width=800,
    height=600 
)

fig.show()

                   Genre  Count
0                  Drama   4394
1                 Comedy   1935
2           Romance Film   1722
3        Black-and-white   1392
4               Thriller   1185
..                   ...    ...
269       Singing cowboy      1
270        Media Studies      1
271          Law & Crime      1
272  Finance & Investing      1
273        Space western      1

[274 rows x 2 columns]


In [38]:
# Filter to only valid rows with release dates
oscar_movies_filtered = oscar_act_movies_all.dropna(subset=['Movie release date'])

def get_movies_and_first_oscar_date(group):
    # Sort movies by release date
    group = group.sort_values(by='Movie release date')
    # Find the first Oscar-winning movie and its date
    first_oscar = group.loc[group['Best Actress Reward']].nsmallest(1, 'Movie release date')
    first_oscar_date = first_oscar['Movie release date'].iloc[0] if not first_oscar.empty else None
    # Count movies before the first Oscar-winning movie
    if pd.notnull(first_oscar_date):
        movies_before = group[group['Movie release date'] < first_oscar_date].shape[0]
    else:
        movies_before = group.shape[0]  # If no Oscar win, count all movies
    
    return pd.Series({'Movies Before First Oscar': movies_before, 'First Oscar Date': first_oscar_date})


movies_before_first_oscar = oscar_movies_filtered.groupby('Actor name').apply(get_movies_and_first_oscar_date).reset_index()
print(movies_before_first_oscar.head(10))
movies_before_first_oscar1 = movies_before_first_oscar[movies_before_first_oscar['Movies Before First Oscar'] < 10]
movies_before_first_oscar2 = movies_before_first_oscar[movies_before_first_oscar['Movies Before First Oscar'] < 30]
movies_before_first_oscar2 = movies_before_first_oscar2[movies_before_first_oscar2['Movies Before First Oscar'] > 15]
print(movies_before_first_oscar2.shape)

fig = px.bar(
    movies_before_first_oscar.sort_values('Movies Before First Oscar', ascending=False),
    x='Actor name',
    y='Movies Before First Oscar',
    title='Number of Movies Before First Oscar per Actress',
    labels={'Actor name': 'Actress', 'Movies Before First Oscar': 'Movies Before First Oscar'},
    hover_data='First Oscar Date',
    text='Movies Before First Oscar',
    color='Movies Before First Oscar',
    color_continuous_scale='Reds'
    
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Actress', tickangle=45),
    yaxis=dict(title='Number of Movies'),
    width=2000,  # Adjust figure width
    height=700  # Adjust figure height
)

# Uncomment this line to display the graph
fig.show()

         Actor name  Movies Before First Oscar  First Oscar Date
0      Adrien Brody                       15.0            2002.0
1         Al Pacino                       15.0            1992.0
2     Alec Guinness                        5.0            1957.0
3      Anna Magnani                        1.0            1955.0
4     Anne Bancroft                        3.0            1962.0
5   Anthony Hopkins                       20.0            1991.0
6        Art Carney                        2.0            1974.0
7    Audrey Hepburn                        0.0            1953.0
8  Barbra Streisand                        0.0            1968.0
9      Ben Kingsley                        0.0            1982.0
(36, 3)


C:\Users\thier\AppData\Local\Temp\ipykernel_7108\2767893020.py:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [29]:
imdb_ratings['Movie name'] = imdb_ratings['Movie name'].str.strip()
filtered_ratings = imdb_ratings[['Movie name', 'Movie release date', 'imdb_score']]
filtered_oscar_actress = oscar_act_movies_all[['Movie name', 'Movie release date', 'Actor name', 'Best Actress Reward']]
merged_data = pd.merge(filtered_ratings, filtered_oscar_actress, how='right', on=['Movie name', 'Movie release date'])


# Extract relevant columns for analysis
rating_evolution = merged_data[['Actor name', 'Movie name', 'Movie release date', 'imdb_score', 'Best Actress Reward']]

# Convert release_year to numeric for proper sorting
rating_evolution['Movie release date'] = pd.to_numeric(rating_evolution['Movie release date'], errors='coerce')


# Filter data for movies released from 1980 onwards
rating_evolution = rating_evolution[rating_evolution['Movie release date'] >= 1980]

# Remove rows with missing or invalid IMDb score
rating_evolution = rating_evolution.dropna(subset=['Movie release date', 'imdb_score'])

# Sort data by actress name and release date
rating_evolution = rating_evolution.sort_values(by=['Actor name', 'Movie release date'])


In [30]:
# Adjust x-axis range to start before the first movie release date
rating_evolution['Movie release date'] = rating_evolution['Movie release date'].astype(int)
x_range = [1975, 2020]  


# Generate a color palette using Plotly's native discrete color scale
actress_movies_count = rating_evolution['Actor name'].value_counts()
relevant_actresses = actress_movies_count[actress_movies_count >= 4].index
relevant_actresses = rating_evolution[rating_evolution['Actor name'].isin(relevant_actresses)]
unique_actresses = relevant_actresses[relevant_actresses['Best Actress Reward'] == True]['Actor name'].unique()

colors = px.colors.qualitative.Plotly  # Or use other palettes like `D3`, `Viridis`, etc.
color_map = {actress: colors[i % len(colors)] for i, actress in enumerate(unique_actresses)}

# Create legend-only traces
legend_traces = [
    go.Scatter(
        x=[None],  # No data points for this trace
        y=[None],  # No data points for this trace
        mode="markers",
        marker=dict(
            size=8,
            color=color_map[actress],
            symbol="circle"  # Always use a circle for the legend
        ),
        name=actress,  # Actress name in the legend
        showlegend=True,
        legendgroup=actress  # Group legend items for the same actress
    )
    for actress in unique_actresses
]


scatter_traces = [
    go.Scatter(
        x=rating_evolution[rating_evolution['Actor name'] == actress]['Movie release date'],
        y=rating_evolution[rating_evolution['Actor name'] == actress]['imdb_score'],
        mode='markers',
        marker=dict(
            size=[15 if reward else 8 for reward in rating_evolution[rating_evolution['Actor name'] == actress]['Best Actress Reward']],
            color=color_map[actress],
            symbol=[
                'star' if reward else 'circle'  # Conditional marker symbol
                for reward in rating_evolution[rating_evolution['Actor name'] == actress]['Best Actress Reward']
            ]
        ),
        name=actress,
        legendgroup=actress,  # Group scatter and line for the same actress
        showlegend=False,  # Show legend for scatter
        hovertext=[
            f"Name: {actress_i}<br>Movie: {movie}<br>IMDb Score: {score}<br>Release Date: {year}<br>Academy Award for Best Actor/Actress: {'Yes' if reward else 'No'}"
            for movie, score, year, reward, actress_i in zip(
                rating_evolution[rating_evolution['Actor name'] == actress]['Movie name'],
                rating_evolution[rating_evolution['Actor name'] == actress]['imdb_score'],
                rating_evolution[rating_evolution['Actor name'] == actress]['Movie release date'],
                rating_evolution[rating_evolution['Actor name'] == actress]['Best Actress Reward'],
                rating_evolution[rating_evolution['Actor name'] == actress]['Actor name']
            )
        ],
        hoverinfo="text",  # Show only hover text
    )
    for actress in unique_actresses
]

line_traces = [
    go.Scatter(
        x=rating_evolution[rating_evolution['Actor name'] == actress]['Movie release date'],
        y=rating_evolution[rating_evolution['Actor name'] == actress]['imdb_score'],
        mode='lines+markers',  # Add both lines and markers
        line=dict(color=color_map[actress]),  # Use the same color as the scatter
        marker=dict(
            size=[15 if reward else 8 for reward in rating_evolution[rating_evolution['Actor name'] == actress]['Best Actress Reward']],
            symbol=[
                'star' if reward else 'circle'  # Conditional marker symbol
                for reward in rating_evolution[rating_evolution['Actor name'] == actress]['Best Actress Reward']
            ]
        ),
        name=actress,
        legendgroup=actress,
        visible=False,  # Start with lines hidden
        hovertext=[
            f"Movie: {movie}<br>IMDb Score: {score}<br>Release Date: {year}<br>Academy Award for Best Actor/Actress : {'Yes' if reward else 'No'}"
            for movie, score, year, reward in zip(
                rating_evolution[rating_evolution['Actor name'] == actress]['Movie name'],
                rating_evolution[rating_evolution['Actor name'] == actress]['imdb_score'],
                rating_evolution[rating_evolution['Actor name'] == actress]['Movie release date'],
                rating_evolution[rating_evolution['Actor name'] == actress]['Best Actress Reward']
            )
        ],
        hoverinfo="text"  # Show only hover text
    )
    for actress in unique_actresses
]

# Combine scatter and line traces
traces = legend_traces + scatter_traces + line_traces

# Create dropdown buttons
buttons = [
    {
        "label": "All",
        "method": "update",
        "args": [
            {"visible": [True]*len(legend_traces) + [True] * len(scatter_traces) + [False] * len(line_traces)},  # Show scatter, hide all lines
            {"title": "Evolution of IMDb Scores for Movies Starring Oscar-Winning Actresses (All Actresses)"}
        ]
    }
]

buttons += [
    {
        "label": actress,
        "method": "update",
        "args": [
            {"visible": [False]*len(legend_traces) + [False] * len(scatter_traces) + [actress == a for a in unique_actresses]},
            {"title": f"Evolution of IMDb Scores for Movies Starring {actress}"}
        ]
    }
    for actress in unique_actresses
]

# Create layout
layout = go.Layout(
    title="Evolution of IMDb Scores for Movies Starring Oscar-Winning Actresses/Actors Before and After the Reward(s)",
    xaxis=dict(title="Release Year", range=x_range),
    yaxis=dict(title="IMDb Score"),
    updatemenus=[
        {
            "buttons": buttons,
            "direction": "down",
            "showactive": True,
        }
    ],
    height=700,
    width=1200,
    showlegend=True  # Ensure legend is visible
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show figure
fig.show()

In [31]:
# First, let's get the genres for each actor
genre_by_actor = oscar_act_movies_all.groupby('Actor name')['Movie genres'].apply(lambda x: ','.join([str(g) for g in x if pd.notna(g)])).reset_index()

# Split the genres string and explode to get one row per genre
genre_by_actor['Movie genres'] = genre_by_actor['Movie genres'].str.split(',')
genre_by_actor = genre_by_actor.explode('Movie genres')

# Clean up genres by stripping whitespace and removing empty strings
genre_by_actor['Movie genres'] = genre_by_actor['Movie genres'].str.strip()
genre_by_actor = genre_by_actor[genre_by_actor['Movie genres'] != '']

# Define genre mappings to group similar genres
genre_mappings = {
    'Drama': ['Drama', 'Family Drama', 'Comedy-drama', 'Melodrama', 'Docudrama'],
    'Comedy': ['Comedy', 'Romantic comedy', 'Comedy-drama', 'Black comedy'],
    'Action/Adventure': ['Action', 'Adventure', 'Action/Adventure', 'Thriller', 'War film'],
    'Crime': ['Crime Fiction', 'Crime film', 'Detective fiction', 'Film noir'],
    'Historical': ['Period piece', 'Historical fiction', 'Biography', 'History'],
    'Romance': ['Romance Film', 'Romantic drama', 'Romantic comedy'],
    'Musical': ['Musical', 'Music film', 'Opera film', 'Concert film'],
    'Horror': ['Horror', 'Supernatural horror', 'Slasher', 'Psychological horror'],
    'Western': ['Western', 'Spaghetti Western', 'Contemporary Western'],
    'SciFi/Fantasy': ['Science Fiction', 'Fantasy', 'Superhero film', 'Space opera'],
    'Documentary': ['Documentary', 'Docudrama', 'Documentary drama'],
    'Animation': ['Animation', 'Anime', 'Stop motion', 'Computer animation'],
    'Other': [] # Will catch any genres not matched above
}

# Map genres to their groups
def map_genre(genre):
    for group, genres in genre_mappings.items():
        if any(g.lower() in genre.lower() for g in genres):
            return group
    return 'Other'

genre_by_actor['Movie genres'] = genre_by_actor['Movie genres'].apply(map_genre)

# Count genres for each actor
genre_counts = genre_by_actor.groupby(['Actor name', 'Movie genres']).size().reset_index(name='count')

# Filter to only include genres that appear frequently
min_appearances = 5
frequent_genres = genre_counts.groupby('Movie genres')['count'].sum().reset_index()
frequent_genres = frequent_genres[frequent_genres['count'] >= min_appearances]['Movie genres']
genre_counts = genre_counts[genre_counts['Movie genres'].isin(frequent_genres)]

# Create a grouped bar chart
fig = px.bar(genre_counts, 
             x='Actor name', 
             y='count',
             color='Movie genres',
             title='Major Film Genres by Oscar-Winning Actor',
             labels={'count': 'Number of Movies', 'Actor name': 'Actor'},
             height=800)

# Update layout for better readability
fig.update_layout(
    xaxis={'categoryorder':'total descending'},
    showlegend=True,
    legend_title='Genre',
    barmode='stack',
    xaxis_tickangle=-45
)

fig.show()

In [32]:
oscar_actress_info_new = pd.merge(winning_actors_info, oscar_act_movies_new, left_on='page_id', right_on='Actress id')
oscar_actors_info_new = pd.merge(winning_actors_info, oscar_act_movies_new, left_on='page_id', right_on='Actors id')
oscar_act_info_new = pd.concat([oscar_actors_info_new, oscar_actress_info_new], axis=0)

oscar_act_info_new = pd.merge(oscar_act_info_new, character[['Wikipedia movie ID', 'Actor name', 'Actor age at movie release', 'Actor date of birth', 'Actor ethnicity', 'Freebase actor ID']], on=['Wikipedia movie ID', 'Actor name'])
oscar_act_info_new['Movie release date'] = oscar_act_info_new['Movie release date'].apply(standardize_date_format)
print(oscar_act_info_new.shape)
oscar_act_info_new.columns

(177, 30)


Index(['page_id', 'wikidata_id', 'sexLabel', 'nativeLanguageLabel',
       'countryOfCitizenshipLabel', 'ethnicGroupLabel', 'Actor name',
       'Actors id', 'Actress id', 'dataset_id', 'runtime', 'languages',
       'countries', 'genres', 'reviewScores', 'awardsReceived',
       'awardsNominated', 'capitalCost', 'release date', 'nbOscarReceived',
       'nbOscarNominated', 'Wikipedia movie ID', 'Movie name',
       'Movie release date', 'Movie genres', 'Movie box office revenue',
       'Actor age at movie release', 'Actor date of birth', 'Actor ethnicity',
       'Freebase actor ID'],
      dtype='object')

In [33]:
oscar_act_new_movies_all = character[character['Freebase actor ID'].isin(oscar_act_info_new['Freebase actor ID'])]
oscar_act_new_movies_all = oscar_act_new_movies_all.drop(columns=['Freebase movie ID', 'Movie release date'])
oscar_act_new_movies_all = oscar_act_new_movies_all.merge(new_film_dataset, on='Wikipedia movie ID', how='left')
oscar_act_new_movies_all['Movie release year'] = oscar_act_new_movies_all['Movie release date'].apply(standardize_date_format)
print('Shape:', oscar_act_new_movies_all.shape)
print('Columns:', oscar_act_new_movies_all.columns)
print('Date boundary:', oscar_act_new_movies_all['Movie release year'].min(), '-', oscar_act_new_movies_all['Movie release year'].max())

# Create a set of unique (Actor name, Wikipedia movie ID) combinations for rewards
best_act_movies = set(
    zip(oscar_act_info_new['Actor name'], oscar_act_info_new['Wikipedia movie ID'])
)

# Add a new column indicating if the movie is a Best Actress Reward
oscar_act_new_movies_all['Best Actress Reward'] = oscar_act_new_movies_all.apply(
    lambda row: (row['Actor name'], row['Wikipedia movie ID']) in best_act_movies,
    axis=1
)



Shape: (6869, 28)
Columns: Index(['Wikipedia movie ID', 'Character name', 'Actor date of birth',
       ' Actor gender', 'Actor height)', 'Actor ethnicity', 'Actor name',
       'Actor age at movie release', 'Freebase character',
       'Freebase character ID', 'Freebase actor ID', 'dataset_id', 'runtime',
       'languages', 'countries', 'genres', 'reviewScores', 'awardsReceived',
       'awardsNominated', 'capitalCost', 'release date', 'nbOscarReceived',
       'nbOscarNominated', 'Movie name', 'Movie release date', 'Movie genres',
       'Movie box office revenue', 'Movie release year'],
      dtype='object')
Date boundary: 1909.0 - 2015.0


In [34]:
imdb_ratings['Movie name'] = imdb_ratings['Movie name'].str.strip()

filtered_ratings = imdb_ratings[['Movie name', 'Movie release date', 'imdb_score']]
filtered_oscar_act_new = oscar_act_info_new[['Movie name', 'Movie release date', 'Actor name', 'Movie box office revenue']]

merged_data = pd.merge(filtered_ratings, filtered_oscar_act_new, on=['Movie name', 'Movie release date'])

rating_box_office = merged_data[['Actor name', 'Movie name', 'Movie release date', 'imdb_score', 'Movie box office revenue']]
rating_box_office['Movie release date'] = pd.to_numeric(rating_box_office['Movie release date'], errors='coerce')

rating_box_office = rating_box_office[rating_box_office['Movie release date'] >= 1980]
rating_box_office = rating_box_office.dropna(subset=['Movie box office revenue', 'imdb_score'])

In [35]:
new_film_dataset_oscar = new_film_dataset[new_film_dataset['Wikipedia movie ID'].isin(oscar_winning_films['Page ID'])]
new_film_dataset_oscar['Movie release date'] = pd.to_datetime(new_film_dataset_oscar['Movie release date'], errors='coerce').dt.year

filtered_ratings = imdb_ratings[['Movie name', 'Movie release date', 'imdb_score']]
filtered_oscar_act_new = new_film_dataset_oscar[['Movie name', 'Movie release date', 'Movie box office revenue', 'nbOscarNominated', 'nbOscarReceived']]

merged_data = pd.merge(filtered_ratings, filtered_oscar_act_new, on=['Movie name', 'Movie release date'])

rating_box_office = merged_data[['Movie name', 'Movie release date', 'imdb_score', 'Movie box office revenue', 'nbOscarNominated', 'nbOscarReceived']]

rating_box_office = rating_box_office[rating_box_office['Movie release date'] >= 1980]
rating_box_office = rating_box_office.dropna(subset=['Movie box office revenue', 'imdb_score'])

C:\Users\thier\AppData\Local\Temp\ipykernel_7108\544752972.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
# Filter out rows with missing or invalid data in critical columns
rating_box_office = rating_box_office.dropna(subset=['Movie release date', 'imdb_score', 'Movie box office revenue', 'nbOscarNominated'])
rating_box_office = rating_box_office[rating_box_office['Movie name'] != 'Titanic']
# Ensure `nbOscarNominated` is numeric
rating_box_office['nbOscarNominated'] = pd.to_numeric(
    rating_box_office['nbOscarNominated'], errors='coerce'
)


# Create a bubble chart using Plotly Express
fig = px.scatter(
    rating_box_office,
    x='Movie release date',
    y='imdb_score',
    size='Movie box office revenue',
    hover_name='Movie name',  # Adjust column name if necessary
    title="Bubble Graph: IMDb Score vs. Movie Release Date (Bubble Size: Box Office Revenue)",
    color='nbOscarNominated',
    labels={
        'Movie release date': 'Release Date',
        'imdb_score': 'IMDb Score',
        'Movie box office revenue': 'Box Office Revenue'
    },
    category_orders={'nbOscarNominated': sorted(rating_box_office['nbOscarNominated'].unique())}  # Sort legend
)

# Show the plot
fig.show()